[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neo4j-field/call-transcripts-automation/blob/main/ui-notebook/ui.ipynb)


# KG + AI Customer Service Assistant

This notebook replicates the Next.js UI for the Call Transcripts Automation project using **Gradio**.
It connects to a Neo4j knowledge graph and uses Azure OpenAI to simulate customer calls,
generate recommendations, and visualize process maps.

**How to use:**
1. Run all cells in order
2. Fill in your credentials in Cell 2 (use Colab Secrets or a `.env` file)
3. Click **New Call** to start a simulated customer conversation
4. Type responses as the employee and click **Send**
5. Use the sidebar for recommendations, suggested responses, and process maps

In [ ]:
!pip install -q gradio neo4j langchain-openai langchain-core openai pyvis python-dotenv

In [ ]:
import os

# In Google Colab, store credentials in Secrets (key icon in left sidebar).
# Locally, create a .env file in this directory.
try:
    from google.colab import userdata
    for key in [
        "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT",
        "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD", "NEO4J_DATABASE",
        "OPENAI_API_KEY",
    ]:
        try:
            os.environ[key] = userdata.get(key)
        except Exception:
            pass
    print("Loaded credentials from Colab Secrets")
except ImportError:
    from dotenv import load_dotenv
    load_dotenv()
    print("Loaded credentials from .env file")


In [ ]:
from neo4j import GraphDatabase
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from pydantic import BaseModel

# --- Neo4j ---
driver = GraphDatabase.driver(
    os.environ["NEO4J_URI"],
    auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]),
)
driver.verify_connectivity()
DATABASE = os.environ.get("NEO4J_DATABASE", "neo4j")
print(f"Connected to Neo4j at {os.environ['NEO4J_URI']}")

# --- Embedder ---
embedder = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    model="text-embedding-3-small",
    openai_api_version="2025-01-01-preview",
    dimensions=128,
)

# --- LLMs ---
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment="gpt-4o",
    temperature=0.8,
    openai_api_version="2025-01-01-preview",
)

llm_mini = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment="gpt-4o-mini",
    temperature=0,
    openai_api_version="2025-01-01-preview",
)

# Structured output for validation
class ValidationResult(BaseModel):
    validated: bool

validation_llm = llm_mini.with_structured_output(ValidationResult)

print("LLM and embedder initialized")


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

DEFAULT_K = 7
COMMENT_EMBEDDINGS_INDEX_NAME = "commentEmbeddings"


# ── 4a: sim_user ─────────────────────────────────────────────────────────────

def sim_user(comment_id, messages):
    """Simulate a customer response.

    Args:
        comment_id: Current comment ID in the sample conversation, or None to start.
        messages: List of dicts with 'role' and 'content' keys.

    Returns:
        Tuple of (comment_dict, new_comment_id).
    """
    # Step 1: Validate / resolve comment ID
    with driver.session(database=DATABASE) as session:
        if not comment_id:
            result = session.run(
                "MATCH (c:Call) "
                "WITH c ORDER BY rand() LIMIT 1 "
                "MATCH (c)-[:FIRST]->(comm) "
                "RETURN comm"
            )
            record = result.single()
            comment_id = record["comm"]["id"]
        elif messages:
            validation_messages = [
                (
                    "system",
                    "Review the messages and determine if the user has addressed "
                    "the most recent comment from their counterparty. "
                    "View the final assistant message and then look if the final "
                    "user message has adequately addressed it. "
                    "Note that in some situations, the employee will need to ask "
                    "for information — this can be considered a temporary resolution "
                    "and the conversation can progress. "
                    "If it has, return validated: true. Otherwise return validated: false.",
                ),
            ] + [(m["role"], m["content"]) for m in messages]

            result = validation_llm.invoke(validation_messages)

            if result.validated:
                query_result = session.run(
                    "OPTIONAL MATCH (comm:Comment {id: $commentId})"
                    "-[:NEXT]->()-[:NEXT]->(nextCustomerComm:Comment) "
                    "RETURN nextCustomerComm AS nextComm",
                    commentId=comment_id,
                )
                record = query_result.single()
                next_comm = record["nextComm"]
                if next_comm:
                    comment_id = next_comm["id"]

    # Step 2: Get comment sample
    with driver.session(database=DATABASE) as session:
        result = session.run(
            "MATCH (comm:Comment {id: $commentId}) "
            "MATCH (comm)-[:OBSERVED_STATE]->(:Observation:State)"
            "-[:IS_PROCESS_ELEMENT]->(:ProcessElement:State)"
            "<-[:IS_PROCESS_ELEMENT]-(:Observation:State)"
            "<-[:OBSERVED_STATE]-(similarComment:Comment) "
            "WITH comm, similarComment "
            "ORDER BY vector.similarity.cosine(comm.embedding, similarComment.embedding) DESC "
            "LIMIT toInteger($k) "
            "RETURN collect(similarComment.content) AS commentSample",
            commentId=comment_id,
            k=DEFAULT_K,
        )
        record = result.single()
        comment_sample = record["commentSample"] if record else []

    # Step 3: Generate customer message
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are simulating a customer engaged with a customer service rep.\n"
            "You can see the comments up to this point in the message history, if any.\n"
            "From these, you can get a sense for the personality you're impersonating "
            "and the flow of the conversation.\n"
            "You should produce the next message in the conversation.\n"
            "Make a comment that makes sense in the context of the conversation.\n"
            "Additionally, see below a list of sample comments.\n"
            "These are comments that are similar in intent to the comment you should "
            "write next.\n\n"
            "Comment Sample:\n{comment_sample}",
        ),
        ("placeholder", "{messages}"),
    ])

    chain = prompt | llm
    formatted_messages = [(m["role"], m["content"]) for m in messages] if messages else []

    response = chain.invoke({
        "messages": formatted_messages,
        "comment_sample": "\n".join(comment_sample),
    })

    return {"role": "assistant", "content": response.content}, comment_id


# ── 4b: sidebar_recommendation ───────────────────────────────────────────────

def sidebar_recommendation(comments):
    """Generate a sidebar recommendation for the employee.

    Args:
        comments: List of dicts with 'role' and 'content'.

    Returns:
        Recommendation text.
    """
    if not comments:
        return "*No conversation yet.*"

    # Step 1: Embed last comment
    embedding = embedder.embed_documents([comments[-1]["content"]])[0]

    # Step 2: Retrieve action paths
    with driver.session(database=DATABASE) as session:
        result = session.run(
            "CALL db.index.vector.queryNodes($commentEmbeddings, $k, $embedding) "
            "YIELD node AS similarComment "
            "MATCH (similarComment)-[:OBSERVED_STATE]->()-[:IS_PROCESS_ELEMENT]->(pe) "
            "MATCH (pe)-[r:ACTION_SELECTION]->(action) "
            "WITH r.probability AS probability, action "
            "ORDER BY probability DESC "
            "LIMIT 1 "
            "MATCH (res:ProcessElement:Resolution) "
            "MATCH p = shortestPath((action)-[:TRANSITION|ACTION_SELECTION|PROCESS_END*]->(res)) "
            "RETURN "
            "  reduce(prob = probability, rel IN relationships(p) | prob * rel.probability) AS historicalProbability, "
            "  [node IN nodes(p) | node.name] AS path "
            "ORDER BY historicalProbability DESC "
            "LIMIT 5",
            embedding=embedding,
            k=DEFAULT_K,
            commentEmbeddings=COMMENT_EMBEDDINGS_INDEX_NAME,
        )
        paths = [
            {"path": record["path"], "probability": record["historicalProbability"]}
            for record in result
        ]

    # Step 3: Generate recommendation
    comments_text = "\n".join(
        f"{'Employee' if c['role'] == 'user' else 'Customer'}: {c['content']}"
        for c in comments[:-1]
    )
    latest = f"{'Employee' if comments[-1]['role'] == 'user' else 'Customer'}: {comments[-1]['content']}"
    if comments_text:
        comments_text += f"\n\nLatest comment:\n{latest}"
    else:
        comments_text = latest

    paths_text = "\n".join(", ".join(p["path"]) for p in paths)

    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a conversational agent that is designed to help telecom "
            "customer service representatives talk to customers.\n"
            "Your job is to recommend the next action to the representative.\n"
            "You will see the list of comments up to this point as well as your "
            "own suggestion history to the rep.\n"
            "You will also get a list of recommended action paths the rep could "
            "follow, ordered by historical probability.\n"
            "Please consider these paths, consider the sentiment and context, "
            "and provide a recommendation.\n"
            "Put the recommendation into your own words and also advise on tone "
            "and approach.\n"
            "Only rely on the context you've been provided — don't make up any "
            "new information.\n"
            "Reply in 2 sentences or less. Do not recommend language. Just give advice.\n\n"
            "Comments:\n{comments}\n\nAction paths:\n{paths}",
        ),
    ])

    chain = prompt | llm
    response = chain.invoke({"comments": comments_text, "paths": paths_text})
    return response.content


# ── 4c: suggested_response ────────────────────────────────────────────────────

def suggested_response(comments):
    """Generate one suggested response for the employee.

    Args:
        comments: List of dicts with 'role' and 'content'.

    Returns:
        Suggested response text.
    """
    if not comments:
        return ""

    # Step 1: Embed last comment
    embedding = embedder.embed_documents([comments[-1]["content"]])[0]

    # Step 2: Retrieve action and resolutions
    with driver.session(database=DATABASE) as session:
        result = session.run(
            "CALL db.index.vector.queryNodes($commentEmbeddings, $k, $embedding) "
            "YIELD node AS similarComment "
            "MATCH (similarComment)-[:OBSERVED_STATE]->()-[:IS_PROCESS_ELEMENT]->(pe) "
            "MATCH (pe)-[r:ACTION_SELECTION]->(action) "
            "WITH r.probability AS probability, action "
            "ORDER BY probability DESC "
            "LIMIT 1 "
            "MATCH (res:ProcessElement:Resolution) "
            "MATCH p = shortestPath((action)-[:TRANSITION|ACTION_SELECTION|PROCESS_END*]->(res)) "
            "WITH action, "
            "  reduce(prob = probability, rel IN relationships(p) | prob * rel.probability) AS historicalProbability, "
            "  [node IN nodes(p) | node] AS path "
            "ORDER BY historicalProbability DESC "
            "LIMIT 3 "
            "RETURN action, collect(path[-1]) AS resolutions",
            embedding=embedding,
            k=DEFAULT_K,
            commentEmbeddings=COMMENT_EMBEDDINGS_INDEX_NAME,
        )
        record = result.single()
        if not record:
            return "Unable to generate suggestion — no matching data found."
        action = record["action"]
        resolutions = record["resolutions"]

    # Step 3: Get action samples
    with driver.session(database=DATABASE) as session:
        result = session.run(
            "MATCH (action:ProcessElement:Action {id: $actionId}) "
            "WITH action "
            "UNWIND $resolutionIds AS resolutionId "
            "MATCH (resolution:ProcessElement:Resolution {id: resolutionId}) "
            "MATCH (action)<-[:IS_PROCESS_ELEMENT]-(:Observation)"
            "<-[:OBSERVED_ACTION]-(comm:Comment)"
            "<-[:NEXT*]-()<-[:FIRST]-(:Call)"
            "-[:OBSERVED_RESOLUTION]->(resolution) "
            "WITH resolution, comm "
            "ORDER BY rand() "
            "WITH resolution, collect(comm.content) AS contents "
            "RETURN resolution, contents[..2] AS contents",
            actionId=action["id"],
            resolutionIds=[r["id"] for r in resolutions],
        )
        contents = [
            {"resolution": record["resolution"], "contents": record["contents"]}
            for record in result
        ]

    # Step 4: Generate
    conversation_text = "\n".join(
        f"{'Employee' if c['role'] == 'user' else 'Customer'}: {c['content']}"
        for c in comments[:-1]
    )
    latest = f"{'Employee' if comments[-1]['role'] == 'user' else 'Customer'}: {comments[-1]['content']}"
    if conversation_text:
        conversation_text += f"\n\nLatest comment:\n{latest}"
    else:
        conversation_text = latest

    suggestions_text = "\n\n".join(
        f"Outcome: {item['resolution']['name']}\n"
        + "\n".join(f"Comment: {c}" for c in item["contents"])
        for item in contents
    )

    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a conversational agent that is designed to help telecom "
            "customer service representatives talk to customers.\n"
            "Your job is to suggest a response in the conversation based on the "
            "context provided.\n"
            "This context includes both the conversation history as well as a "
            "list of comments made by the rep in similar situations.\n"
            "Your response should ideally be 2-3 sentences long and should be "
            "helpful and informative, but if it needs to be longer be as concise "
            "as possible.\n"
            "Respect the tone and sentiment of the call and respect the "
            "disposition of the customer.\n"
            "We want to be as helpful as possible to the customer and also have "
            "an eye towards positive outcomes for us.\n"
            "The example comments are organized by projected outcome, so decide "
            "what the best likely outcome is and respond accordingly.\n\n"
            "Conversation history:\n{conversation}\n\n{suggestions}",
        ),
    ])

    chain = prompt | llm
    response = chain.invoke({
        "conversation": conversation_text,
        "suggestions": suggestions_text,
    })
    return response.content


# ── 4d: get_process_map ──────────────────────────────────────────────────────

def get_process_map(comment_content):
    """Get process-map nodes and relationships for a comment.

    Args:
        comment_content: The text of the comment to analyse.

    Returns:
        Tuple of (nodes_list, rels_list).
    """
    embedding = embedder.embed_documents([comment_content])[0]

    with driver.session(database=DATABASE) as session:
        result = session.run(
            "CALL db.index.vector.queryNodes($commentEmbeddings, $k, $embedding) "
            "YIELD node AS similarComment "
            "MATCH (similarComment)-[:OBSERVED_STATE]->()-[:IS_PROCESS_ELEMENT]->(pe) "
            "MATCH (pe)-[r:ACTION_SELECTION]->(action) "
            "WITH r.probability AS probability, action "
            "ORDER BY probability DESC "
            "LIMIT 1 "
            "MATCH (res:ProcessElement:Resolution) "
            "MATCH p = shortestPath((action)-[:TRANSITION|ACTION_SELECTION|PROCESS_END*]->(res)) "
            "WITH nodes(p) AS nodes, relationships(p) AS rels "
            "LIMIT 5 "
            "WITH collect(nodes) AS allNodes, collect(rels) AS allRels "
            "WITH apoc.coll.toSet(apoc.coll.flatten(allNodes)) AS nodes, "
            "     apoc.coll.toSet(apoc.coll.flatten(allRels)) AS rels "
            "RETURN "
            "  [n IN nodes | { "
            "    id: n.id + '', "
            "    description: n.description, "
            "    name: n.name, "
            "    label: [lbl IN labels(n) WHERE lbl <> 'ProcessElement' | lbl][0] "
            "  }] AS nodes, "
            "  [r IN rels | { "
            "    id: elementId(r), "
            "    from: startNode(r).id + '', "
            "    to: endNode(r).id + '', "
            "    probability: r.probability, "
            "    type: type(r) "
            "  }] AS rels",
            embedding=embedding,
            k=DEFAULT_K,
            commentEmbeddings=COMMENT_EMBEDDINGS_INDEX_NAME,
        )
        record = result.single()
        if not record:
            return [], []
        return record["nodes"], record["rels"]


print("Agent functions loaded")

In [ ]:
from pyvis.network import Network
import base64


def render_process_map(nodes, rels):
    """Render process-map nodes and relationships as an interactive HTML graph."""
    if not nodes:
        return "<p style='text-align:center;color:#888;'>No process map data available.</p>"

    net = Network(
        height="500px",
        width="100%",
        directed=True,
        notebook=True,
        cdn_resources="remote",
    )
    net.barnes_hut(gravity=-3000, central_gravity=0.3, spring_length=200)

    color_map = {"State": "#93c5fd", "Action": "#6ee7b7", "Resolution": "#fca5a5"}

    for node in nodes:
        color = color_map.get(node.get("label"), "#d4d4d8")
        net.add_node(
            node["id"],
            label=node.get("name", node["id"]),
            title=node.get("description") or node.get("name", ""),
            color=color,
            shape="dot",
            size=25,
            font={"size": 14},
        )

    for rel in rels:
        label = rel.get("type", "")
        prob = rel.get("probability")
        if prob is not None:
            try:
                label += f" ({float(prob):.0%})"
            except (TypeError, ValueError):
                pass
        net.add_edge(
            rel["from"],
            rel["to"],
            title=label,
            label=label,
            font={"size": 10},
        )

    html = net.generate_html()

    # Inject a legend into the HTML
    legend = (
        '<div style="position:absolute;top:10px;left:10px;background:white;'
        "padding:8px;border-radius:8px;box-shadow:0 2px 4px rgba(0,0,0,0.1);"
        'z-index:100;font-family:sans-serif;font-size:12px;">'
        '<div style="display:flex;align-items:center;gap:6px;margin-bottom:4px;">'
        '<div style="width:14px;height:14px;border-radius:50%;background:#93c5fd;"></div>State</div>'
        '<div style="display:flex;align-items:center;gap:6px;margin-bottom:4px;">'
        '<div style="width:14px;height:14px;border-radius:50%;background:#6ee7b7;"></div>Action</div>'
        '<div style="display:flex;align-items:center;gap:6px;">'
        '<div style="width:14px;height:14px;border-radius:50%;background:#fca5a5;"></div>Resolution</div>'
        "</div>"
    )
    html = html.replace("<body>", f"<body>{legend}")

    b64 = base64.b64encode(html.encode()).decode()
    return f'<iframe src="data:text/html;base64,{b64}" width="100%" height="550px" frameborder="0"></iframe>'


print("Process map renderer loaded")

In [ ]:
import gradio as gr


# ── Event handlers ────────────────────────────────────────────────────────────

def handle_new_call():
    """Start a new simulated customer call."""
    try:
        comment, comment_id = sim_user(None, [])
    except Exception as e:
        return (
            [{"role": "assistant", "content": f"[Error starting call: {e}]"}],
            f"*Error: {e}*",
            None,
            [],
            "",
            "", "", "",
            "",
        )

    messages = [comment]
    chat_history = [{"role": "assistant", "content": comment["content"]}]

    try:
        rec = sidebar_recommendation(messages)
    except Exception as e:
        rec = f"*Error generating recommendation: {e}*"

    return (
        chat_history,
        f"**Recommendation:**\n\n{rec}",
        comment_id,
        messages,
        "",        # clear input
        "", "", "",  # clear suggestions
        "",        # clear process map
    )


def handle_send(user_msg, chat_history, comment_id, messages):
    """Send a user (employee) message and get a customer response."""
    if not user_msg or not user_msg.strip():
        return (
            chat_history or [],
            "*Enter a message first.*",
            comment_id,
            messages or [],
            "",
        )

    messages = list(messages or []) + [{"role": "user", "content": user_msg}]
    chat_history = list(chat_history or []) + [{"role": "user", "content": user_msg}]

    try:
        comment, new_comment_id = sim_user(comment_id, messages)
        messages = messages + [{"role": "assistant", "content": comment["content"]}]
        chat_history = chat_history + [
            {"role": "assistant", "content": comment["content"]}
        ]
    except Exception as e:
        chat_history = chat_history + [
            {"role": "assistant", "content": f"[Error: {e}]"}
        ]
        new_comment_id = comment_id

    try:
        rec = sidebar_recommendation(messages)
    except Exception as e:
        rec = f"*Error: {e}*"

    return (
        chat_history,
        f"**Recommendation:**\n\n{rec}",
        new_comment_id,
        messages,
        "",
    )


def handle_use_suggestion(suggestion, chat_history, comment_id, messages):
    """Use a suggested response as the employee message."""
    if not suggestion or not suggestion.strip():
        return (
            chat_history or [],
            "*No suggestion to use.*",
            comment_id,
            messages or [],
            "",
        )
    return handle_send(suggestion, chat_history, comment_id, messages)


def handle_generate_suggestions(messages):
    """Generate 3 suggested responses."""
    if not messages:
        return "No conversation yet.", "No conversation yet.", "No conversation yet."
    results = []
    for _ in range(3):
        try:
            results.append(suggested_response(messages))
        except Exception as e:
            results.append(f"Error: {e}")
    return results[0], results[1], results[2]


def handle_refresh_map(messages):
    """Refresh the process map based on the latest comment."""
    if not messages:
        return "<p style='text-align:center;color:#888;'>Start a conversation first.</p>"
    try:
        nodes, rels = get_process_map(messages[-1]["content"])
        return render_process_map(nodes, rels)
    except Exception as e:
        return f"<p style='color:red;'>Error generating process map: {e}</p>"


# ── Build the Gradio UI ──────────────────────────────────────────────────────

with gr.Blocks(title="KG + AI Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# KG + AI Customer Service Assistant")

    # State
    comment_id_state = gr.State(value=None)
    messages_state = gr.State(value=[])

    with gr.Row():
        # ---- Left column: Chat ----
        with gr.Column(scale=1):
            chatbot = gr.Chatbot(
                type="messages",
                height=500,
                label="Conversation  (You = Employee, AI = Customer)",
            )
            msg_input = gr.Textbox(
                placeholder="Type your response as the employee...",
                label="Your Message",
                lines=2,
            )
            with gr.Row():
                send_btn = gr.Button("Send", variant="primary")
                new_call_btn = gr.Button("New Call", variant="secondary")

        # ---- Right column: Sidebar ----
        with gr.Column(scale=1):
            recommendation = gr.Markdown(
                value="*Click **New Call** to start a conversation.*",
                label="Recommendation",
            )

            with gr.Accordion("Details", open=True):
                with gr.Tab("Suggested Responses"):
                    gen_suggestions_btn = gr.Button(
                        "Generate Suggestions", variant="primary"
                    )
                    suggestion_1 = gr.Textbox(
                        label="Suggestion 1", interactive=False, lines=3
                    )
                    use_1_btn = gr.Button("Use Response 1", size="sm")
                    suggestion_2 = gr.Textbox(
                        label="Suggestion 2", interactive=False, lines=3
                    )
                    use_2_btn = gr.Button("Use Response 2", size="sm")
                    suggestion_3 = gr.Textbox(
                        label="Suggestion 3", interactive=False, lines=3
                    )
                    use_3_btn = gr.Button("Use Response 3", size="sm")

                with gr.Tab("Process Map"):
                    refresh_map_btn = gr.Button(
                        "Refresh Process Map", variant="primary"
                    )
                    process_map_html = gr.HTML(
                        value="<p style='text-align:center;color:#888;'>"
                        "Click <b>Refresh</b> after starting a conversation.</p>"
                    )

    # ---- Event wiring ----
    new_call_outputs = [
        chatbot, recommendation, comment_id_state, messages_state,
        msg_input, suggestion_1, suggestion_2, suggestion_3, process_map_html,
    ]
    new_call_btn.click(fn=handle_new_call, inputs=[], outputs=new_call_outputs)

    send_outputs = [
        chatbot, recommendation, comment_id_state, messages_state, msg_input,
    ]
    send_btn.click(
        fn=handle_send,
        inputs=[msg_input, chatbot, comment_id_state, messages_state],
        outputs=send_outputs,
    )
    msg_input.submit(
        fn=handle_send,
        inputs=[msg_input, chatbot, comment_id_state, messages_state],
        outputs=send_outputs,
    )

    for btn, sug in [
        (use_1_btn, suggestion_1),
        (use_2_btn, suggestion_2),
        (use_3_btn, suggestion_3),
    ]:
        btn.click(
            fn=handle_use_suggestion,
            inputs=[sug, chatbot, comment_id_state, messages_state],
            outputs=send_outputs,
        )

    gen_suggestions_btn.click(
        fn=handle_generate_suggestions,
        inputs=[messages_state],
        outputs=[suggestion_1, suggestion_2, suggestion_3],
    )

    refresh_map_btn.click(
        fn=handle_refresh_map,
        inputs=[messages_state],
        outputs=[process_map_html],
    )

demo.launch()